# 2.4 Maxwell's Equations

[Peter Monk: "Finite Elements for Maxwell's Equations"]

## Magnetostatic field generated by a permanent magnet

magnetic flux $B$, magnetic field $H$, given magnetization $M$:

$$
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Div}{div}
B = \mu (H + M), \quad \Div B = 0, \quad \Curl H = 0
$$

Introducing a vector-potential $A$ such that $B = \Curl A$, and putting equations together we get

$$
\Curl \mu^{-1} \Curl A = \Curl M
$$

In weak form: Find $A \in H(\Curl)$ such that

$$
\int \mu^{-1} \Curl A \Curl v = \int M \Curl v \qquad  \forall \, v \in H(\Curl)
$$

Usually, the permeability $\mu$ is given as 
$\mu = \mu_r \mu_0$, with $\mu_0 = 4 \pi 10^{-7}$ the permeability of vacuum.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import *

Geometric model and meshing of a bar magnet:

In [ ]:
box = OrthoBrick(Pnt(-3,-3,-3),Pnt(3,3,3)).bc("outer")
magnet = Cylinder(Pnt(-1,0,0),Pnt(1,0,0), 0.3) * OrthoBrick(Pnt(-1,-3,-3),Pnt(1,3,3))
air = box - magnet

geo = CSGeometry()
geo.Add (air.mat("air"), transparent=True) 
geo.Add (magnet.mat("magnet").maxh(1), col=(0.3,0.3,0.1))
geo.Draw()
mesh = Mesh(geo.GenerateMesh(maxh=2, curvaturesafety=1))
mesh.Curve(3)

In [ ]:
mesh.GetMaterials(), mesh.GetBoundaries()

Define space, forms and preconditioner.

* To obtain a regular system matrix, we regularize by adding a very small $L_2$ term.
* We solve magnetostatics, so we can gauge by adding and arbitrary gradient field. A cheap possibility is to delete all basis-functions which are gradients (flag 'nograds')

In [ ]:
fes = HCurl(mesh, order=3, dirichlet="outer", nograds=True)
print ("ndof =", fes.ndof)
u,v = fes.TnT()

from math import pi
mu0 = 4*pi*1e-7
mur = mesh.MaterialCF({"magnet" : 1000}, default=1)

a = BilinearForm(fes)
a += 1/(mu0*mur)*curl(u)*curl(v)*dx + 1e-8/(mu0*mur)*u*v*dx
c = Preconditioner(a, "bddc")

f = LinearForm(fes)
mag = mesh.MaterialCF({"magnet" : (1,0,0)}, default=(0,0,0))
f += mag*curl(v) * dx("magnet")

Assemble system and setup preconditioner using task-parallelization:

In [ ]:
with TaskManager():
    a.Assemble()
    f.Assemble()

Finally, declare GridFunction and solve by preconditioned CG iteration:

In [ ]:
gfu = GridFunction(fes)
with TaskManager():
    solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=c.mat)

In [ ]:
# the vector potential is not supposed to look nice
Draw (gfu, mesh, "vector-potential", draw_surf=False, clipping=True)

Draw (curl(gfu), mesh, "B-field", draw_surf=False, clipping=True)
Draw (1/(mu0*mur)*curl(gfu)-mag, mesh, "H-field", draw_surf=False, clipping=True)